In [6]:
import sys, os, json, glob, math, shutil, numpy as np
import open3d as o3d


In [7]:
from pathlib import Path

def read_ply_as_points(path: str) -> o3d.geometry.PointCloud:
    """Read a PLY; if it's a triangle mesh, sample points; otherwise load as point cloud."""
    mesh = o3d.io.read_triangle_mesh(path)
    if mesh.has_triangles():
        mesh.compute_vertex_normals()
        area = mesh.get_surface_area()
        n = max(int(area * 1000), 200_000)  # heuristic; increase if too sparse
        pcd = mesh.sample_points_poisson_disk(number_of_points=n, use_triangle_normal=True)
        return pcd
    pcd = o3d.io.read_point_cloud(path)
    if len(pcd.points) == 0:
        raise RuntimeError(f"No geometry found in {path}")
    return pcd

def preprocess_for_reg(pcd: o3d.geometry.PointCloud, voxel: float):
    """Downsample + normals + FPFH features for registration."""
    p_down = pcd.voxel_down_sample(voxel)
    p_down.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=voxel*2, max_nn=30))
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        p_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel*5, max_nn=100)
    )
    return p_down, fpfh

def register_pair(source_p: o3d.geometry.PointCloud, target_p: o3d.geometry.PointCloud, voxel: float):
    """Coarse RANSAC on FPFH, then refine with point-to-plane ICP."""
    src_d, src_f = preprocess_for_reg(source_p, voxel)
    tgt_d, tgt_f = preprocess_for_reg(target_p, voxel)
    dist = voxel * 1.5

    ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        src_d, tgt_d, src_f, tgt_f, True, dist,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False), 4, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(dist)
        ],
        o3d.pipelines.registration.RANSACConvergenceCriteria(400000, 500)
    )
    icp = o3d.pipelines.registration.registration_icp(
        source_p, target_p, max_correspondence_distance=voxel,
        init=ransac.transformation,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane()
    )
    return icp.transformation, {"fitness": icp.fitness, "rmse": icp.inlier_rmse}

def transform_and_stack(point_clouds, transforms):
    """Apply transforms to each cloud and stack into one point cloud (keeps colors if present)."""
    pts, cols = [], []
    for pcd, T in zip(point_clouds, transforms):
        p = o3d.geometry.PointCloud(pcd)  # copy
        p.transform(T)
        pts.append(np.asarray(p.points))
        if p.has_colors(): cols.append(np.asarray(p.colors))
    all_pts = np.vstack(pts)
    merged = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(all_pts))
    if cols and len(cols) == len(pts):
        merged.colors = o3d.utility.Vector3dVector(np.vstack(cols))
    return merged

def poisson_to_mesh(pcd: o3d.geometry.PointCloud, depth=10, scale=1.1, target_tris=None):
    """Reconstruct a mesh from points (Screened Poisson) and optionally decimate."""
    if not pcd.has_normals():
        pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=40))
        pcd.orient_normals_consistent_tangent_plane(20)

    mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=depth, scale=scale, linear_fit=False
    )
    mesh = mesh.crop(pcd.get_axis_aligned_bounding_box())
    mesh.remove_unreferenced_vertices()
    mesh.compute_vertex_normals()

    if target_tris and len(mesh.triangles) > target_tris:
        mesh = mesh.simplify_quadric_decimation(target_number_of_triangles=int(target_tris))
        mesh.compute_vertex_normals()
    return mesh

def write_glb_from_mesh(mesh: o3d.geometry.TriangleMesh, out_path: str):
    import trimesh  # lazy import so trimesh is optional
    verts = np.asarray(mesh.vertices)
    faces = np.asarray(mesh.triangles)
    tmesh = trimesh.Trimesh(vertices=verts, faces=faces, process=False)
    data = trimesh.exchange.gltf.export_glb(tmesh)
    with open(out_path, "wb") as f:
        f.write(data)


In [10]:
# Put your PLY paths here.
# Option A: Glob a folder
# ply_paths = sorted(glob.glob("/path/to/folder/*.ply"))

# Option B: List manually
ply_paths = [
    "./merge_test/realistic_1.ply",
    "./merge_test/realistic_2.ply",
    # add more...
]

out_merged_ply = "./merge_test/merged.ply"        # point-cloud output
write_glb      = True                         # also create a GLB mesh?
out_glb        = "./merge_test/merged.glb"

need_registration = False   # set True if files are NOT pre-aligned
voxel_reg      = 0.03       # meters; used only when need_registration=True
voxel_final    = 0.01       # meters; final downsample/dedup for merged point cloud

# GLB meshing params (only used if write_glb=True)
poisson_depth  = 10         # 10..12 typical; higher = more detail/slower
poisson_scale  = 1.1        # trims "balloon"; try 1.05..1.1
target_tris    = 150_000    # triangle budget for colliders


In [11]:
assert len(ply_paths) >= 2, "Need at least two PLYs"
clouds = [read_ply_as_points(p) for p in ply_paths]
print(f"Loaded {len(clouds)} point clouds")

if need_registration:
    transforms = [np.eye(4)]
    ref = clouds[0]
    for idx, mov in enumerate(clouds[1:], start=1):
        T, stats = register_pair(mov, ref, voxel=voxel_reg)
        print(f"[{Path(ply_paths[idx]).name}] ICP fitness={stats['fitness']:.3f} rmse={stats['rmse']:.4f}")
        transforms.append(T)
else:
    transforms = [np.eye(4) for _ in clouds]

merged = transform_and_stack(clouds, transforms)
print("Merged points before voxel:", np.asarray(merged.points).shape[0])

if voxel_final and voxel_final > 0:
    merged = merged.voxel_down_sample(voxel_final)

print("Merged points after voxel:", np.asarray(merged.points).shape[0])
ok = o3d.io.write_point_cloud(out_merged_ply, merged, write_ascii=False, compressed=True)
print("Wrote merged PLY:", ok, out_merged_ply)


[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
Loaded 2 point clouds
Merged points before voxel: 4000000
Merged points after voxel: 2454914
Wrote merged PLY: True ./merge_test/merged.ply


: 

In [ ]:
mesh = poisson_to_mesh(
    merged, depth=poisson_depth, scale=poisson_scale, target_tris=target_tris
)
print("Triangles:", len(mesh.triangles))
try:
    write_glb_from_mesh(mesh, out_glb)
    print("Wrote GLB:", out_glb)
except ModuleNotFoundError:
    print("Install trimesh to export GLB:  pip install trimesh")
